In [ ]:
!pip install requests librosa numpy matplotlib tensorflow

import numpy as np
import os
import requests
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf



In [ ]:
!pip install audioread

In [ ]:
def fetch_bird_data(species, num_recordings=10):
    url = f'https://www.xeno-canto.org/api/2/recordings?query={species}'
    response = requests.get(url)
    data = response.json()

    recordings = []
    for recording in data['recordings'][:num_recordings]:
        recordings.append({
            'id': recording['id'],
            'gen': recording['gen'],
            'sp': recording['sp'],
            'file': recording['file'],
            'type': recording['type']
        })
    return recordings


In [ ]:
bird_species = ['parus major', 'cyanistes caeruleus', 'fringilla coelebs']
recordings = []
for species in bird_species:
    fetched_data = fetch_bird_data(species, num_recordings=10)
    print(f"Fetched {len(fetched_data)} recordings for {species}")
    recordings.extend(fetched_data)


Fetched 10 recordings for parus major
Fetched 10 recordings for cyanistes caeruleus
Fetched 10 recordings for fringilla coelebs


In [ ]:
def download_audio(recording, save_dir='audio_files'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # URL'nin "https:" ile başladığından emin ol
    file_url = recording['file']
    if not file_url.startswith('https://'):
        file_url = f"https:{file_url}"

    file_path = os.path.join(save_dir, f"{recording['id']}.mp3")

    response = requests.get(file_url)
    if response.status_code == 200:
        with open(file_path, 'wb') as f:
            f.write(response.content)
        return file_path
    else:
        print(f"Failed to download {file_url}")
        return None


In [ ]:
import librosa
import numpy as np
import os

def extract_features(file_path, max_pad_len=128):
    print(f"Extracting features from {file_path}")
    try:
        audio, sample_rate = librosa.load(file_path, sr=None, res_type='kaiser_fast')
        if audio is None:
            raise ValueError("Audio loading failed. Check the file path or format.")

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=128)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)


        # Zero padding veya truncation
        if mel_spec_db.shape[1] < max_pad_len:
            pad_width = max_pad_len - mel_spec_db.shape[1]
            mel_spec_db = np.pad(mel_spec_db, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mel_spec_db = mel_spec_db[:, :max_pad_len]

        return mel_spec_db
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [ ]:
features = []
labels = []

for recording in recordings:
    try:
        file_path = download_audio(recording)
        if file_path:
            mel_spec_db = extract_features(file_path)
            if mel_spec_db is not None:
                print('mel_spec_db')
                features.append(mel_spec_db)
                labels.append(f"{recording['gen']} {recording['sp']}")
    except Exception as e:
        print(f"Error processing {recording['id']}: {e}")

print(f"Extracted features from {len(features)} recordings")


Extracting features from audio_files/913706.mp3
mel_spec_db
Extracting features from audio_files/913021.mp3
mel_spec_db
Extracting features from audio_files/912848.mp3
mel_spec_db
Extracting features from audio_files/912846.mp3
mel_spec_db
Extracting features from audio_files/912624.mp3
mel_spec_db
Extracting features from audio_files/912622.mp3
mel_spec_db
Extracting features from audio_files/911422.mp3
mel_spec_db
Extracting features from audio_files/911184.mp3
mel_spec_db
Extracting features from audio_files/910517.mp3
mel_spec_db
Extracting features from audio_files/910504.mp3
mel_spec_db
Extracting features from audio_files/913157.mp3
mel_spec_db
Extracting features from audio_files/912950.mp3
mel_spec_db
Extracting features from audio_files/912827.mp3
mel_spec_db
Extracting features from audio_files/912196.mp3
mel_spec_db
Extracting features from audio_files/910883.mp3
mel_spec_db
Extracting features from audio_files/910195.mp3
mel_spec_db
Extracting features from audio_files/906

In [ ]:

features = np.array(features)
print(features.shape)
labels = np.array(labels)


(30, 128, 128)


In [ ]:

if len(features) > 0:
    print(len(features))
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)

    X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

    # Boyutları yeniden şekillendirme
    X_train = X_train[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(128, 128, 1)),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

    # Modeli değerlendirme
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Test Accuracy: {accuracy}')
else:
    print("No features extracted, check your data and processing steps.")

30
Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 9.5946 - accuracy: 0.3158 - val_loss: 249.0050 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 1s 556ms/step - loss: 175.9003 - accuracy: 0.4211 - val_loss: 232.6814 - val_accuracy: 0.6000
Epoch 3/10
1/1 [==============================] - 1s 519ms/step - loss: 297.5028 - accuracy: 0.2632 - val_loss: 151.5596 - val_accuracy: 0.6000
Epoch 4/10
1/1 [==============================] - 1s 502ms/step - loss: 188.2034 - accuracy: 0.2632 - val_loss: 70.5980 - val_accuracy: 0.6000
Epoch 5/10
1/1 [==============================] - 1s 530ms/step - loss: 59.8101 - accuracy: 0.2632 - val_loss: 120.2703 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 481ms/step - loss: 73.0248 - accuracy: 0.4211 - val_loss: 98.0562 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 1s 530ms/step - loss: 56.1912 - accuracy: 0.4211 - val_loss: 50.0757 - val_accur

In [ ]:
from IPython.display import Audio, display

def predict_bird_species(file_path):
    print(f"Predicting species for {file_path}")
    mel_spec_db = extract_features(file_path)
    if mel_spec_db is None:
        return None, "Error: Failed to extract features"

    # Boyut ekleyerek veya yeniden şekillendirerek uygun hale getirme
    mel_spec_db = mel_spec_db[np.newaxis, ..., np.newaxis]

    predictions = model.predict(mel_spec_db)
    predicted_label = label_encoder.inverse_transform([np.argmax(predictions)])

    # Ses dosyasını çalma
    display(Audio(file_path))

    # Gerçek kuş türü
    real_species = os.path.basename(file_path).split('.')[0]  # Örneğin dosya adı 910195.mp3 ise '910195' olarak alıyoruz

    return predicted_label[0], real_species

# Yeni bir ses kaydında kuş türü tahmini
new_file_path = '/content/audio_files/910195.mp3'
predicted_species, real_species = predict_bird_species(new_file_path)
print(f'Tahmin Edilen Kuş Türü: {predicted_species}')
print(f'Gerçek Kuş Türü: {real_species}')


Predicting species for /content/audio_files/910195.mp3
Extracting features from /content/audio_files/910195.mp3
1/1 [==============================] - 0s 34ms/step


Tahmin Edilen Kuş Türü: Cyanistes caeruleus
Gerçek Kuş Türü: 910195
